# Time Series Models Demo: Energy Consumption Forecasting

This notebook provides a quick side-by-side comparison of different time series models for energy consumption forecasting.

## Models Compared:
1. **ARIMA** - Classical statistical approach
2. **xLSTM** - Deep learning with explainability
3. **AutoGluon** - AutoML ensemble approach

## Evaluation Metrics:
- MAE (Mean Absolute Error)
- RMSE (Root Mean Squared Error)
- Runtime (seconds)

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Data Generation

We generate synthetic energy consumption data with:
- **Trend**: Linear upward trend
- **Seasonality**: Annual seasonal pattern
- **Noise**: Random fluctuations

In [ ]:
def generate_energy_data(n_samples=1000):
    """Generate synthetic energy consumption data with trend and seasonality"""
    date_rng = pd.date_range(start='2020-01-01', periods=n_samples, freq='D')
    df = pd.DataFrame(date_rng, columns=['timestamp'])
    
    # Create trend component
    trend = np.linspace(0, 20, n_samples)
    
    # Create seasonal component (annual cycle)
    season = 10 * np.sin(2 * np.pi * np.arange(n_samples) / 365.25)
    
    # Create random noise
    noise = np.random.normal(0, 5, n_samples)
    
    df['energy_consumption'] = 100 + trend + season + noise
    df.set_index('timestamp', inplace=True)
    return df

# Generate data
np.random.seed(42)  # For reproducibility
df = generate_energy_data()

# Split into train and test
test_size = 30
train_data = df[:-test_size]
test_data = df[-test_size:]

print(f"Dataset shape: {df.shape}")
print(f"Training samples: {len(train_data)}")
print(f"Test samples: {len(test_data)}")

# Visualize the data
plt.figure(figsize=(14, 5))
plt.plot(train_data.index, train_data['energy_consumption'], label='Training Data', alpha=0.7)
plt.plot(test_data.index, test_data['energy_consumption'], label='Test Data', color='orange', alpha=0.7)
plt.title('Energy Consumption Data', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Energy Consumption (kWh)')
plt.legend()
plt.tight_layout()
plt.show()

## 2. Model 1: ARIMA

**Why ARIMA?**
- Interpretable statistical model
- Fast training on univariate data
- Good baseline for comparison
- Handles trend and seasonality explicitly

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima

print("Training ARIMA model...")
start_time = time.time()

# Auto-select best parameters
model_auto = auto_arima(train_data['energy_consumption'], 
                        start_p=0, start_q=0, max_p=5, max_q=5, m=7,
                        start_P=0, seasonal=True, d=1, D=1, 
                        trace=False, error_action='ignore',
                        suppress_warnings=True, stepwise=True)

# Fit model
model_arima = ARIMA(train_data['energy_consumption'], order=model_auto.order)
results_arima = model_arima.fit()

# Predict
forecast_arima = results_arima.forecast(steps=test_size)

arima_time = time.time() - start_time

# Calculate metrics
arima_mae = mean_absolute_error(test_data['energy_consumption'], forecast_arima)
arima_rmse = np.sqrt(mean_squared_error(test_data['energy_consumption'], forecast_arima))

print(f"✓ ARIMA trained in {arima_time:.2f} seconds")
print(f"  Best order: {model_auto.order}")
print(f"  MAE: {arima_mae:.3f}")
print(f"  RMSE: {arima_rmse:.3f}")

## 3. Model 2: xLSTM (Simplified LSTM)

**Why xLSTM?**
- Captures complex non-linear patterns
- Learns from sequential dependencies
- More flexible than statistical models
- Can incorporate multiple features

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

print("Training xLSTM model...")
start_time = time.time()

# Prepare data
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data[['energy_consumption']])
test_scaled = scaler.transform(test_data[['energy_consumption']])

# Create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:(i + seq_length)])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

seq_length = 30
X_train, y_train = create_sequences(train_scaled, seq_length)

# Build model
model_lstm = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(seq_length, 1)),
    LSTM(50, activation='relu'),
    Dense(1)
])

model_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Train
history = model_lstm.fit(X_train, y_train, epochs=20, batch_size=32, 
                         validation_split=0.1, verbose=0)

# Predict on test data
# Use last seq_length points from training to start prediction
last_sequence = train_scaled[-seq_length:]
forecast_lstm = []

for _ in range(test_size):
    next_pred = model_lstm.predict(last_sequence.reshape(1, seq_length, 1), verbose=0)
    forecast_lstm.append(next_pred[0, 0])
    last_sequence = np.roll(last_sequence, -1, axis=0)
    last_sequence[-1] = next_pred

forecast_lstm = scaler.inverse_transform(np.array(forecast_lstm).reshape(-1, 1)).flatten()

lstm_time = time.time() - start_time

# Calculate metrics
lstm_mae = mean_absolute_error(test_data['energy_consumption'], forecast_lstm)
lstm_rmse = np.sqrt(mean_squared_error(test_data['energy_consumption'], forecast_lstm))

print(f"✓ xLSTM trained in {lstm_time:.2f} seconds")
print(f"  MAE: {lstm_mae:.3f}")
print(f"  RMSE: {lstm_rmse:.3f}")

## 4. Model 3: AutoGluon-TimeSeries

**Why AutoGluon?**
- Automated model selection and ensembling
- Combines multiple model types
- Minimal hyperparameter tuning needed
- Often achieves state-of-the-art results

In [ ]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

print("Training AutoGluon model...")
start_time = time.time()

# Prepare data in AutoGluon format
train_ag = pd.DataFrame({
    'timestamp': train_data.index,
    'target': train_data['energy_consumption'].values,
    'item_id': ['energy'] * len(train_data)
})

train_ag = TimeSeriesDataFrame.from_data_frame(train_ag, id_column='item_id', timestamp_column='timestamp')

# Train predictor
predictor = TimeSeriesPredictor(
    prediction_length=test_size,
    target='target',
    verbosity=0
)

predictor.fit(
    train_ag,
    presets='fast_training',  # Use fast preset for demo
    time_limit=60  # Limit training time
)

# Predict
forecast_ag = predictor.predict(train_ag)
forecast_autogluon = forecast_ag['mean'].values

autogluon_time = time.time() - start_time

# Calculate metrics
autogluon_mae = mean_absolute_error(test_data['energy_consumption'], forecast_autogluon)
autogluon_rmse = np.sqrt(mean_squared_error(test_data['energy_consumption'], forecast_autogluon))

print(f"✓ AutoGluon trained in {autogluon_time:.2f} seconds")
print(f"  MAE: {autogluon_mae:.3f}")
print(f"  RMSE: {autogluon_rmse:.3f}")

## 5. Results Comparison

### Summary Metrics Table

In [ ]:
# Create comparison DataFrame
results = pd.DataFrame({
    'Model': ['ARIMA', 'xLSTM', 'AutoGluon'],
    'MAE': [arima_mae, lstm_mae, autogluon_mae],
    'RMSE': [arima_rmse, lstm_rmse, autogluon_rmse],
    'Runtime (s)': [arima_time, lstm_time, autogluon_time]
})

# Add relative performance
results['MAE (% of best)'] = (results['MAE'] / results['MAE'].min() * 100).round(1)
results['Runtime (% of fastest)'] = (results['Runtime (s)'] / results['Runtime (s)'].min() * 100).round(1)

print("\n" + "="*70)
print("MODEL COMPARISON SUMMARY")
print("="*70)
print(results.to_string(index=False))
print("="*70)

# Highlight best performer
best_mae_idx = results['MAE'].idxmin()
best_rmse_idx = results['RMSE'].idxmin()
fastest_idx = results['Runtime (s)'].idxmin()

print(f"\n✓ Best MAE: {results.loc[best_mae_idx, 'Model']}")
print(f"✓ Best RMSE: {results.loc[best_rmse_idx, 'Model']}")
print(f"✓ Fastest: {results.loc[fastest_idx, 'Model']}")

### Visual Comparison

In [ ]:
# Plot all forecasts together
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Plot 1: All forecasts overlaid
ax = axes[0, 0]
ax.plot(test_data.index, test_data['energy_consumption'], 
        label='Actual', color='black', linewidth=2, marker='o', markersize=4)
ax.plot(test_data.index, forecast_arima, 
        label=f'ARIMA (MAE={arima_mae:.2f})', alpha=0.7, linewidth=2)
ax.plot(test_data.index, forecast_lstm, 
        label=f'xLSTM (MAE={lstm_mae:.2f})', alpha=0.7, linewidth=2)
ax.plot(test_data.index, forecast_autogluon, 
        label=f'AutoGluon (MAE={autogluon_mae:.2f})', alpha=0.7, linewidth=2)
ax.set_title('Forecast Comparison', fontsize=12, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Energy Consumption (kWh)')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: MAE Comparison
ax = axes[0, 1]
bars = ax.bar(results['Model'], results['MAE'], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
ax.set_title('Mean Absolute Error (Lower is Better)', fontsize=12, fontweight='bold')
ax.set_ylabel('MAE')
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.2f}', ha='center', va='bottom', fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# Plot 3: RMSE Comparison
ax = axes[1, 0]
bars = ax.bar(results['Model'], results['RMSE'], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
ax.set_title('Root Mean Squared Error (Lower is Better)', fontsize=12, fontweight='bold')
ax.set_ylabel('RMSE')
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.2f}', ha='center', va='bottom', fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# Plot 4: Runtime Comparison
ax = axes[1, 1]
bars = ax.bar(results['Model'], results['Runtime (s)'], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
ax.set_title('Training Runtime (Lower is Better)', fontsize=12, fontweight='bold')
ax.set_ylabel('Time (seconds)')
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1f}s', ha='center', va='bottom', fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('results/model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Comparison plot saved to 'results/model_comparison.png'")

## 6. Key Takeaways

### Design Choices & Rationale:

1. **Test Size (30 days)**: Chosen to balance evaluation robustness with computational efficiency
2. **Sequence Length (30 for LSTM)**: One month of history provides sufficient context for patterns
3. **Model Selection**:
   - **ARIMA**: Baseline for interpretability and speed
   - **xLSTM**: Deep learning for complex pattern capture
   - **AutoGluon**: State-of-the-art automated approach

### Evaluation Strategy:
- **MAE**: Easier to interpret (average error in kWh)
- **RMSE**: Penalizes large errors more heavily
- **Runtime**: Important for production deployment

### Trade-offs:
- **Speed vs Accuracy**: ARIMA is fastest but may sacrifice accuracy
- **Interpretability vs Performance**: Neural networks are less interpretable
- **Complexity vs Maintainability**: AutoGluon automates complexity but is a black box

---

**Next Steps**: For production deployment, consider:
1. Hyperparameter tuning for the best-performing model
2. Cross-validation for more robust evaluation
3. Feature engineering (weather, holidays, etc.)
4. Ensemble methods combining multiple models